##Trilateration - KiNEXOn

In [ ]:
import time
import random
import os
import math
import numpy as np
import pandas as pd
from sympy import symbols, Eq, solve, solve_poly_system, roots
import multiprocessing as mp
import threading

In [ ]:
#Field data

xe=np.zeros(4)
ye=np.zeros(4)

xe[0]=0
ye[0]=0

xe[1]=100
ye[1]=0

xe[2]=0
ye[2]=60    

xe[3]=100
ye[3]=60

speed_max=10

f=20

dt=1/f

d_err=0.3

a,b=symbols('a,b',Positive=True,real=True)

t=900

cwd=os.getcwd()
#print(cwd)

In [ ]:
#POSITION PROCESSOR

def position():    
    d=np.zeros(4)
    start_position=time.time()
    print (start_position)    
    p=1    
    while p<t+1:
        file_p=cwd+"\\Range\\"+"time-"+str(p)+".dat"
        data = pd.read_csv(file_p, header=None,sep='\t').values             
        d=np.mean(data, axis=0)
        loc1=np.zeros([3,2])
        loc2=np.zeros([2,2])
        loc3=np.zeros(2)
               
        eq_1 = Eq(((a-xe[0])**2+(b-ye[0])**2-d[0]**2),0)    
        for i in range (1,4):            
            eq_2 = Eq(((a-xe[i])**2+(b-ye[i])**2-d[i]**2),0)            
            loc=np.array(solve((eq_1, eq_2),(a,b),real=True))           
            
            j=np.size(loc)           
            if j==0:
                loc1[i-1]=[0,0]
            elif j==2:
                loc1[i-1]=loc                
            else:
                root1=loc[0,:]
                root2=loc[1,:]                               
                if root1[0]<0 or root1[0]>100 or root1[1]<0 or root1[1]>60:
                    loc1[i-1]=root2                
                elif root2[0]<0 or root2[0]>100 or root2[1]<0 or root2[1]>60:
                    loc1[i-1]=root1
                else:
                    distance1=math.sqrt((root1[0]-xe[0])**2+(root1[1]-ye[0])**2)-d[0]
                    distance2=math.sqrt((root2[0]-xe[0])**2+(root2[1]-ye[0])**2)-d[0]
                    if abs(distance1)<abs(distance2):
                        loc1[i-1]=root1                        
                    else:
                        loc1[i-1]=root2                
                       
        eq_1 = Eq(((a-xe[1])**2+(b-ye[1])**2-d[1]**2),0)
        for i in range (2,4):
            eq_2 = Eq(((a-xe[i])**2+(b-ye[i])**2-d[i]**2),0)            
            loc=np.array(solve((eq_1, eq_2),(a,b),real=True))            

            j=np.size(loc)
            if j==0:
                loc2[i-2]=[0,0]
            elif  j==2:
                loc2[i-2]=loc                
            else:
                root1=loc[0,:]
                root2=loc[1,:]            
                if root1[0]<0 or root1[0]>100 or root1[1]<0 or root1[1]>60:
                    loc2[i-2]=root2                             
                elif root2[0]<0 or root2[0]>100 or root2[1]<0 or root2[1]>60:
                    loc2[i-2]=root1            
                else:
                    distance1=math.sqrt((root1[0]-xe[0])**2+(root1[1]-ye[0])**2)-d[0]
                    distance2=math.sqrt((root2[0]-xe[0])**2+(root2[1]-ye[0])**2)-d[0]
                    if abs(distance1)<abs(distance2):
                        loc2[i-2]=root1                        
                    else:
                        loc2[i-2]=root2
                 
        eq_1 = Eq(((a-xe[2])**2+(b-ye[2])**2-d[2]**2),0)
        eq_2 = Eq(((a-xe[3])**2+(b-ye[3])**2-d[3]**2),0)        
        loc=np.array(solve((eq_1, eq_2),(a,b),real=True))
        
        j=np.size(loc)        
        if j==0:
            loc3=[0,0]
        elif j==2:
            loc3=loc            
        else:
            root1=loc[0,:]
            root2=loc[1,:]            
            if root1[0]<0 or root1[0]>100 or root1[1]<0 or root1[1]>60:
                loc3=root2
            elif root2[0]<0 or root2[0]>100 or root2[1]<0 or root2[1]>60:
                loc3=root1
            else:
                distance1=math.sqrt((root1[0]-xe[0])**2+(root1[1]-ye[0])**2)-d[0]
                distance2=math.sqrt((root2[0]-xe[0])**2+(root2[1]-ye[0])**2)-d[0]
                if abs(distance1)<abs(distance2):
                    loc3=root1
                else:
                    loc3=root2       
                
        loc_p=np.row_stack((loc1,loc2,loc3))
        loc_p = loc_p[~np.all(loc_p == 0, axis=1)]        
        file_pos=cwd+"//Position//"+"Position-"+str(p)+".dat"
        np.savetxt(file_pos, loc_p, delimiter ="\t")

        file=open('p_xy',"a")
        pos=np.median(loc_p, axis=0)
        file.write(str(pos[0])+"\t"+str(pos[1])+"\n")
        file.close()
        p=p+1
        time.sleep(1-(time.time()-start_position) % 1)    
    print ('exited position at',time.time()-start_position) 


In [ ]:
#RANGE GENERATOR 

def rangegenerator():    
    start_range=time.time()
    print(start_range)
    d=np.zeros([20,4])
    p_original=np.zeros([20,2])
    #Initial Position
    x=50
    y=30
    ft=0
    n=0    
    while n<t*20:
        n=n+1
        ds=dt*speed_max
        dx=random.uniform(-ds,ds)
        dy=random.uniform(-(ds-dx),(ds-dx))
        
        x_t=x+dx
        if x_t<0: 
            x_t=0.1            
        elif x_t>100:            
            x_t=99.9      
        
        y_t=y+dy
        if y_t<0:            
            y_t=0.1            
        elif y_t>60:            
            y_t=59.9

        c=n%20
        for j in range(4):
            d_err=random.uniform(-0.3,0.3)                
            d[c,j]=np.sqrt((x_t-xe[j])**2+(y_t-ye[j])**2)+d_err
                  
        x=x_t
        y=y_t
        for j in range(2):
            p_original[c,0]=x
            p_original[c,1]=y
                   
        if (c==0):
            ft=ft+1
            filename=cwd+"//Range//"+"time-"+str(ft)+".dat"
            np.savetxt(filename, d, delimiter ="\t")            
            d[:,:]=0
            file=open('p_original',"a")
            pos=np.mean(p_original, axis=0)
            file.write(str(pos[0])+"\t"+str(pos[1])+"\n")
            file.close()
            
        time.sleep(dt-(time.time()-start_range) % dt)      
    print ('exited range at',time.time()-start_range)    


In [ ]:
def main():
    os.mkdir('Range')
    os.mkdir('Position')
    p1 =threading.Thread(target=rangegenerator)
    p1.start()
    time.sleep(2)
    p2 = threading.Thread(target=position)
    p2.start()
    
main()

